# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import pandas as pd
import os
import pickle
from sqlalchemy import create_engine
from sklearn.multioutput import MultiOutputClassifier
import nltk 
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
from sklearn.pipeline import Pipeline 
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier  
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM messages", engine)


In [3]:
#Preview dataset
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
X = df.message.values #Extract messages
y = df.iloc[:,4:] #Extract classification results 

### 2. Write a tokenization function to process your text data

In [5]:
def tokenize(text):
    '''
    Function to split message text into tokens and returns root form of the words
    INPUT: message text (string type)
    OUTPUT: list of words (strings) in root form 
    '''
    #normailze text
    normalized_text=text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    #convert to tokens
    tokens = word_tokenize(normalized_text)
    
    #remove all stopwords
    words = [t for t in tokens if t not in stopwords.words("english")]
    
    #find root form of words
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for w in words:
        clean_tok = lemmatizer.lemmatize(w).strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
#Logistic Regression Pipeline
logreg_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(LogisticRegression()))
    ])

#Support Vector Machine Pipeline
svm_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(SVC()))
    ])

#Naive Bayes Multinomial Pipeline
nb_pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(MultinomialNB()))
    ])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
#Split into testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y)

#Fit the pipelines
logreg_pipeline.fit(X_train, y_train)
svm_pipeline.fit(X_train, y_train)
nb_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...ssifier(estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def print_classification_metrics(y_test, y_pred):
    '''
    Function that prints metrics for a model
    '''
    i=0
    
    for col in y_test:
        print("Feature: ",col)
        print(classification_report(y_test[col], y_pred[:, i]))
        i +=1
    accuracy = (y_pred == y_test.values).mean()
    print("Model Accuracy: ",accuracy)



In [9]:
#Logistic Regression Model Predictions and Results
logreg_y_pred = logreg_pipeline.predict(X_test)
print_classification_metrics(y_test, logreg_y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.72      0.42      0.53      1498
          1       0.85      0.95      0.90      5056

avg / total       0.82      0.83      0.81      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.91      0.97      0.94      5473
          1       0.80      0.53      0.64      1081

avg / total       0.89      0.90      0.89      6554

Feature:  offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6517
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3844
          1       0.77      0.67      0.72      2710

avg / total       0.78      0.78      0.78      6554

Feature:  medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
#Support Vector Machine Predictions and Results
svm_y_pred = svm_pipeline.predict(X_test)
print_classification_metrics(y_test, svm_y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1498
          1       0.77      1.00      0.87      5056

avg / total       0.60      0.77      0.67      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.84      1.00      0.91      5473
          1       0.00      0.00      0.00      1081

avg / total       0.70      0.84      0.76      6554

Feature:  offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6517
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.59      1.00      0.74      3844
          1       0.00      0.00      0.00      2710

avg / total       0.34      0.59      0.43      6554

Feature:  medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [11]:
#Naive Bayes Multinomial Predictions and Results
nb_y_pred = nb_pipeline.predict(X_test)
print_classification_metrics(y_test, nb_y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.82      0.09      0.17      1498
          1       0.79      0.99      0.88      5056

avg / total       0.79      0.79      0.72      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.87      0.99      0.93      5473
          1       0.86      0.23      0.36      1081

avg / total       0.87      0.87      0.83      6554

Feature:  offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6517
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.76      0.85      0.80      3844
          1       0.75      0.62      0.68      2710

avg / total       0.76      0.76      0.75      6554

Feature:  medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
#We got the best accuracy on the Logistic Regression model, so we will apply grid search to it
#Check available parameters
logreg_pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7fb47e1866a8>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
             intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
             penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
             verbose=0, warm_start=False),
              n_jobs=1))],
 'vect': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtyp

In [15]:

parameters = {
    'tfidf__use_idf': (True, False),
    'clf__estimator__random_state': [None,42],
    'clf__estimator__solver':['liblinear','lbfgs']       
}

#Fit a grid search model
cv = GridSearchCV(logreg_pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [16]:
#Naive Bayes Multinomial Gridsearch Predictions and Results
cv.fit(X_train, y_train)
cv_y_pred = cv.predict(X_test)
print_classification_metrics(y_test, cv_y_pred)

Feature:  related
             precision    recall  f1-score   support

          0       0.71      0.47      0.57      1498
          1       0.86      0.94      0.90      5056

avg / total       0.82      0.84      0.82      6554

Feature:  request
             precision    recall  f1-score   support

          0       0.92      0.97      0.94      5473
          1       0.79      0.55      0.65      1081

avg / total       0.90      0.90      0.89      6554

Feature:  offer
             precision    recall  f1-score   support

          0       0.99      1.00      1.00      6517
          1       0.00      0.00      0.00        37

avg / total       0.99      0.99      0.99      6554

Feature:  aid_related
             precision    recall  f1-score   support

          0       0.79      0.86      0.82      3844
          1       0.77      0.67      0.72      2710

avg / total       0.78      0.78      0.78      6554

Feature:  medical_help
             precision    recall  f1-score 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
#As done in cells above

### 9. Export your model as a pickle file

In [19]:
# Create a pickle file for the model
file_name = 'distaster_response_message_classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.